Instalando e importando os requisitos

In [ ]:
%pip install -r ./requirements.txt -q
from app.modelo.resnet50 import Resnet50
from app.modelo.svm import SVM
from app.modelo.imagem_rgb import ImagemRGB
import os
import pandas
import time
import datetime

Abrindo o DataFrame e setando o caminho da imagem de destino e carregando o modelo

In [ ]:
# Carregando o arquivo CSV
## Definindo o caminho do arquivo CSV para quando mudar de diretório não houver problemas
if not 'csv_file_path' in locals(): # Evita definir o caminho do arquivo CSV duas vezes
    csv_file_path = os.path.join(os.getcwd(), "classifications.csv")
data_frame = pandas.read_csv(csv_file_path)

# Definindo o diretório das imagens
if not 'destination_image_dir' in locals(): # Evita definir o diretório das imagens duas vezes
    destination_image_dir = os.path.join(os.getcwd(), "dataset_converted")

# Carregando o modelo
## Mudando o diretório para o diretório app para contornar erro ao importar a inteligência do modelo
if not 'app' in os.getcwd(): # Evita mudar o diretório duas vezes
    os.chdir('./app')
modelo_resnet50 = Resnet50()
modelo_svm = SVM()

In [ ]:
data_frame

Calculando o tempo

In [ ]:
dados_tempo = []
labels_tempo = ['Caminho', 'Algoritmo', 'Tempo', 'Classe']
if not os.path.exists(destination_image_dir):
    os.mkdir(destination_image_dir)
indice = 0
for _, row in data_frame.iterrows():
    classe = row['bethesda_system']
    diretorio_classe = os.path.join(destination_image_dir, classe)
    if not os.path.exists(diretorio_classe):
        os.mkdir(diretorio_classe)
    
    destination_image_path = os.path.join(diretorio_classe, "{}.png".format(row['cell_id']))
    
    if os.path.exists(destination_image_path):
        for _ in range(10):
            # Abrindo a imagem
            imagem = ImagemRGB.from_file(destination_image_path)
            
            # Calculando o tempo do SVM
            inicio = time.time_ns()
            modelo_svm.predict(imagem)
            fim = time.time_ns()
            tempo_svm = (fim - inicio)/10**9
            dados_tempo.append((destination_image_path, 'SVM', tempo_svm, classe))

            # Calculando o tempo do Resnet50
            inicio = time.time_ns()
            modelo_resnet50.predict(imagem)
            fim = time.time_ns()
            tempo_resnet50 = (fim - inicio)/10**9
            dados_tempo.append((destination_image_path, 'Resnet50', tempo_resnet50, classe))

            tempo = pandas.DataFrame(dados_tempo, columns=labels_tempo)
            tempo.to_csv("tempo_{}_{}.csv".format(indice, datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))
            indice = indice + 1
